# Station catalog

This site provides an overview of available solar radiation monitoring stations worldwide and metadata of the stations.

The table is also available to download in CSV form: {download}`SolarStationsOrg-station-catalog.csv`

In [11]:
import pandas as pd
import json
import kgcPy2
from unidecode import unidecode
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)

# Load stations
solarstations = pd.read_csv('../solarstations.csv', dtype={'Tier': str}).fillna('')
esmap_stations = pd.read_csv('../esmap_stations.csv', dtype={'Tier': str}).fillna('')
# Add esmap stations first so they are underneath
stations = pd.concat([solarstations, esmap_stations], axis='rows', ignore_index=True)

with open('country_by_continent.json') as f:
    country_by_continent = json.load(f)

stations['Continent'] = stations['Country'].mapp(country_by_continent)

stations['Koeppen Geiger classification'] = stations.apply(lambda x: kgcPy2.lookupCZ(x['Latitude'], x['Longitude']), axis=1)

kg_climates = {'A': 'Tropical', 'B': 'Dry', 'C': 'Temperate', 'D': 'Continental', 'E': 'Polar', 'O': 'Ocean'}

stations['Koeppen Geiger climate zone'] = stations['Koeppen Geiger classification'].apply(lambda x: kg_climates[x[0]])

# Write file containing all columns, linked to above
stations.to_csv('./SolarStationsOrg-station-catalog.csv', index=False)

# Format station name as hyperlinks if URL is available.
# Do this after the new CSV is written, since we don't want HTML in that.
for index, row in stations.iterrows():
    if row['URL'].startswith('http'):
        stations.loc[index, 'Station full name'] = f'<a href="{row["URL"]}">{row["Station full name"]}</a>'

del stations['URL']  # Remove the URL column to avoid cluttering the site

# Show table with stations (all rows)
show(stations, scrollY="700px", scrollX=True, scrollCollapse=True, paging=False, classes="display", order=[[0, "asc"]],
     columnDefs=[{"className": "dt-left", "targets": "_all"}], maxBytes=0,
     layout={"top1": "searchPanes"}, searchPanes={"layout": "columns-3", "cascadePanes": True, 'columns': [16,3,12], 'i18n': {'title': None}}
)

ModuleNotFoundError: No module named 'kgcPy'